In [1]:
import requests
import psycopg2
import time
from bs4 import BeautifulSoup

/usr/local/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [52]:
import pandas as pd

In [53]:
conn = psycopg2.connect("dbname='movie_recommender' user='postgres' host='localhost' port=5432 password=''")
conn.autocommit = True

In [54]:
def insertRecord(record_to_insert):
    postgres_insert_query = "insert into urls (oldid, newid) values ('%s', '%s')"%record_to_insert
    cur = conn.cursor()
    cur.execute(postgres_insert_query)
    conn.commit()

In [55]:
def updateRecord(record_to_insert):
    postgres_insert_query = "UPDATE urls set url='%s' where id=%d"%record_to_insert
    cur = conn.cursor()
    cur.execute(postgres_insert_query)
    conn.commit()


In [56]:
def hasEmptyUrlRecords():
    sql = "select count(*) from urls where url is null;"
    cur = conn.cursor()
    cur.execute(sql)
    record = None
    for row in cur:
        record = row
    return int(record[0])

In [57]:
def fetchWhereEmptyUrl():
    sql = "select id, newid from urls where url is null limit 1;"
    cur = conn.cursor()
    cur.execute(sql)
    record = None
    for row in cur:
        record = row
    if not record: return (1, '')
    return record

In [67]:
def fetchImageUrl(movie_id):
    url = "https://www.imdb.com/title/%s/"%(movie_id)
    resp = requests.get(url)
    if resp.status_code >= 400:
        print "HTTP Code: ",resp.status_code, url
        return 'None'
    img_url = 'None'
    try:
        bs = BeautifulSoup(resp.text)
        poster_div = bs.findAll("div", {"class": "poster"})[0]
        poster_img = poster_div.findAll('img')[0]
        img_url = poster_img['src']
    except Exception as ex:
        print 'Exception : ', str(ex)
        print url
    return img_url

In [ ]:
resp = requests.get()

In [59]:
id_mapping = pd.read_csv('../data/link.csv')

In [9]:
id_mapping.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [15]:
id_mapping.iloc[0]

movieId         1.0
imdbId     114709.0
tmdbId        862.0
Name: 0, dtype: float64

In [43]:
for i in range(0, id_mapping.shape[0]):
    old_id = str(int(id_mapping.iloc[i][0]))
    new_id = "tt%07d"%int(id_mapping.iloc[i][1])
    insertRecord((old_id, new_id))


In [17]:
id_mapping.iloc[0][1]

114709.0

In [38]:
'%07d'%113845

'0113845'

In [68]:
for i in range(id_mapping.shape[0]):
    rid, imdb_id = fetchWhereEmptyUrl()
    if imdb_id != '':
        img_url = fetchImageUrl(imdb_id)
        time.sleep(0.08)
        if img_url != '':
            updateRecord((img_url, rid))
    if i % 100 == 0:
        print i,

HTTP Code:  404 https://www.imdb.com/title/tt0118114/
0 Exception :  list index out of range
https://www.imdb.com/title/tt0114894/
Exception :  list index out of range
https://www.imdb.com/title/tt0125877/
Exception :  list index out of range
https://www.imdb.com/title/tt0105737/
Exception :  list index out of range
https://www.imdb.com/title/tt0116992/
Exception :  list index out of range
https://www.imdb.com/title/tt0038426/
100 200 300 Exception :  list index out of range
https://www.imdb.com/title/tt0102336/
400 Exception :  list index out of range
https://www.imdb.com/title/tt0116403/
500 Exception :  list index out of range
https://www.imdb.com/title/tt0115548/
Exception :  list index out of range
https://www.imdb.com/title/tt0107314/
600 Exception :  list index out of range
https://www.imdb.com/title/tt0113212/
Exception :  list index out of range
https://www.imdb.com/title/tt0123281/
700 800 900 Exception :  list index out of range
https://www.imdb.com/title/tt0120881/
1000 110

Exception :  list index out of range
https://www.imdb.com/title/tt2519480/
Exception :  list index out of range
https://www.imdb.com/title/tt0113286/
Exception :  list index out of range
https://www.imdb.com/title/tt1587848/
Exception :  list index out of range
https://www.imdb.com/title/tt1937189/
Exception :  list index out of range
https://www.imdb.com/title/tt1290732/
Exception :  list index out of range
https://www.imdb.com/title/tt1555904/
19900 20000 Exception :  list index out of range
https://www.imdb.com/title/tt0134790/
Exception :  list index out of range
https://www.imdb.com/title/tt2429312/
Exception :  list index out of range
https://www.imdb.com/title/tt0493129/
Exception :  list index out of range
https://www.imdb.com/title/tt2780668/
20100 20200 20300 20400 Exception :  list index out of range
https://www.imdb.com/title/tt0087117/
Exception :  list index out of range
https://www.imdb.com/title/tt0443091/
20500 20600 Exception :  list index out of range
https://www.imd

In [10]:
# 8808 tt0464029 

In [11]:
conn.close()